In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
import os

from dataclasses import dataclass
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from pathlib import Path

In [2]:
!make Simulation

g++ -g -O2 -Wall -Wextra -Wshadow -Werror -std=c++20  -c Lattice.cpp -o Lattice.o
g++ -g -O2 -Wall -Wextra -Wshadow -Werror -std=c++20  -c Simulation.cpp -o Simulation.o
g++ -o Simulation HashTable.o Lattice.o Simulation.o -L/usr/local/lib -Wl,-rpath,/usr/local/lib -lgsl -lgslcblas -lm


Following the work of [Lattice Simulations of Nonperturbative Quantum Field Theories](https://inspirehep.net/literature/1386200) by David Schaich, we will perform a lattice QFT simulation of a $\phi^4$ field using the metropolis and the wolff algorithms.

Let's start with the metropolis bits.

# Metropolis

For the metropolis steps, we need to be able to compute the energy of our system in order to determine whether we accept or reject a change.

As we saw in the Isisng model example, in order to meet the more rigurous condition of detailed balance we need to probabilitically accept a change according to

$$
A(\mu \rightarrow \nu)
=
\begin{cases}
1           & \text{if } E_\nu \leq E_\mu \quad(\Delta E \leq 0) \\
e^{-\beta (E_\nu - E_\mu)} = e^{-\beta \Delta E}   & \text{if } E_\nu > E_\mu \quad(\Delta E > 0)
\end{cases}
$$

And this is where we will take a more research-based pedagogy and work backwards - as opposed to sending you off to learn QFT, quantum mechanics, electrodynamics, etc, etc.

The $\phi^4$ theory in four-dimensional Euclidean space has the following expression for its action

$$
S_{E} = 
\int d^4 x_E \mathcal{L}_{E} =
\int d^4 x \left[ 
    \frac{1}{2} \left( \partial_{E_\mu} \phi \right)^2 + \frac{1}{2} \mu_0 \phi^2 + \frac{1}{4} \lambda_0 \phi^4
\right]
$$

And it turns out that in Euclidean space the action, or Euclidean action, has the same units as the energy, so from here on out we will refer to the above as the energy.

Just to start out simple, we will work in 2D first.
And we will:
1. Discretize $\phi$: the continuous function $\phi(x_\mu)$ will be represented by $\phi_n$ for $0 \leq n \leq N$, on an Lattice of zise $N$ and spacing $a$ between points.
2. Discretize the derivatives: for example, $\partial \phi / \partial x$ becomes $\frac{1}{a} \left( \phi_n(x+a,t) - \phi_n(x,t) \right)$
3. Discretize the integral: $\int dx f$ becomes $\sum^{N}_{i} f(n_i) a$

All to define the operations we need in a lattice.